# Extracting sign distributions

This notbook contains code for:
- importing necessary libraries
- loading stored WTs
- creating random sparse networks
- Extracting sign distributions from WTs
- Extracting sign distributions from non-WTs
- saving extracted distributions in files

### Requirements:

In [1]:
# importing necessary libraries and the cnn architecture I defined

from cnn_architecture import CNN2Model
from utils import *

import tensorflow_datasets as tfds
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [2]:
def load_and_prep_cifar(batch_size, shuffle_size):
    # load data set
    (train_ds, test_ds), ds_info = tfds.load(name="cifar10", split=["train","test"], as_supervised=True, with_info=True)
    # tfds.show_examples(train_ds, ds_info)
    
    def prepare_cifar10_data(ds):
        #convert data from uint8 to float32
        ds = ds.map(lambda img, target: (tf.cast(img, tf.float32), target))
        #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
        ds = ds.map(lambda img, target: ((img/128.)-1., target))
        #create one-hot targets
        ds = ds.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
        #cache this progress in memory, as there is no need to redo it; it is deterministic after all
        ds = ds.cache()
        #shuffle, batch, prefetch
        ds = ds.shuffle(shuffle_size).batch(batch_size).prefetch(2)
        #return preprocessed dataset
        return ds
    
    # prepare data
    train_dataset = train_ds.apply(prepare_cifar10_data)
    test_dataset = test_ds.apply(prepare_cifar10_data)
    
    return train_dataset, test_dataset

In [3]:
# extract sign distribution from one layer

def sign_distribution_layer(this_layer, next_layer):
    '''
    columns: 
    ["prune_rate_in", "prune_rate_out", "pos_in", "pos_out", "neg_in", "neg_out", "sign_rate_in", "sign_rate_out"]
    '''
    sign_distr = pd.DataFrame()
    for this_neuron, next_neurons, i in zip(this_layer.T, next_layer, range(np.shape(this_layer)[0])):
        
        this_unpruned = this_neuron[this_neuron != 0.0]
        next_unpruned = next_neurons[next_neurons != 0.0]
        dic = {} 
        dic["pos_in"] = len(this_unpruned[this_unpruned>0])
        dic["pos_out"] = len(next_unpruned[next_unpruned>0])
        dic["neg_in"] = len(this_unpruned[this_unpruned<0])
        dic["neg_out"] = len(next_unpruned[next_unpruned<0])
        if len(this_neuron) > 0:
            dic["prune_rate_in"] = 1 - (len(this_unpruned)/len(this_neuron))
        else:
            dic["prune_rate_in"] = 1
        if len(next_unpruned) > 0:
            dic["prune_rate_out"] = 1 - (len(next_unpruned)/len(next_neurons))
        else:    
            dic["prune_rate_out"] = 1
        if len(this_unpruned) > 0:
            dic["sign_rate_in"] = dic["pos_in"]/len(this_unpruned)
        else:
            dic["sign_rate_in"] = 0
        if len(next_unpruned) > 0:    
            dic["sign_rate_out"] = dic["pos_out"]/len(next_unpruned)
        else:
            dic["sign_rate_out"] = 0                                
        df = pd.DataFrame(data = dic, index = [i])
        sign_distr = pd.concat([sign_distr, df], axis=0)
        
    return sign_distr

In [4]:
# extract sign distribution form two layers

def sign_distribution_layers(layer1, layer2, layer3):
    sign_distr1 = sign_distribution_layer(layer1, layer2)
    sign_distr2 = sign_distribution_layer(layer2, layer3)
    sign_distr1["layer"] = "dense1"
    sign_distr2["layer"] = "dense2"
    return pd.concat([sign_distr1, sign_distr2], axis=0)

## Extracting sign distributions and storing them as files:

In [21]:
# extract sign distributions

def get_sign_distr(dataset, pruning_name, n):

    # make a model to load the weights into
    train_dataset, test_dataset = load_and_prep_cifar(batch_size=60, shuffle_size=512)
    model = CNN2Model()
    model(list(train_dataset)[0][0])
    for i in range(n):
        
        # get WT weights, extract sign distribution and store in file
        model.load_weights(f"1b WTs/WT_{dataset}_{pruning_name}_{i}")    
        weights_wt = model.get_weights()
        sign_distr_wt = sign_distribution_layers(weights_wt[4], weights_wt[6], weights_wt[8])
        sign_distr_wt.to_csv(f'2b Sign distributions/{dataset}_{pruning_name}_{i}_sign_distr.csv', index=False)

2024-05-08 16:19:34.007109: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2024-05-08 16:19:34.007589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


2024-05-08 16:19:37.255520: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open 1b WTs/WT_SVHN_IMP_0: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


In [ ]:
get_sign_distr("SVHN","IMP",10)

### Generating randomly pruned sign distributions

In [5]:
# implement random pruning of a weight matrix according to a pruning rate

def random_pruning(model,pruning_rates):
    
    weights = model.get_weights()
    new_weights = []
    
    for layer, p_rate in zip(weights[::2], pruning_rates):
        shape = np.shape(layer)
        layer = layer.flatten()
        number_to_prune = int(len(layer) * p_rate)
        pruning_indexi = random.sample(range(len(layer)),number_to_prune)
        layer[pruning_indexi] = 0.0
        layer = np.reshape(layer,shape)
        new_weights.append(layer)
        
    weights[::2] = new_weights
    model.set_weights(weights)
    weights = model.get_weights()
    
    return model

In [7]:
# get pruning rates from imp model
train_dataset, test_dataset = load_and_prep_cifar(batch_size=60, shuffle_size=512)
model_cifar_imp = CNN2Model()
model_cifar_imp(list(train_dataset)[0][0])
model_cifar_imp.load_weights("1b WTs/WT_CIFAR_IMP_0")
pruning_rates = get_pruning_rates(model_cifar_imp.get_weights()[::2])

# generate random sparse networks
for i in range(10):
    random_model = CNN2Model()
    random_model(list(train_dataset)[0][0])
    random_model = random_pruning(random_model,pruning_rates)
    weights_rsn = random_model.get_weights()
    sign_distr_random = sign_distribution_layers(weights_rsn[4], weights_rsn[6], weights_rsn[8])
    sign_distr_random.to_csv(f"2b Sign distributions/RSN_{i}_sign_distr.csv", index=False)

conv rate:  0.6794931592039801
dense rate:  0.9176456925675676


### Inspecting the data

In [ ]:
# return the pruning rates of the conv and dense layers
print("conv rate: ", get_pruning_rate(model_cifar_imp.get_weights()[0:4:2]))
print("dense rate: ", get_pruning_rate(model_cifar_imp.get_weights()[4::2]))

In [23]:
sign_distr_random.head() 

,pos_in,pos_out,neg_in,neg_out,prune_rate_in,prune_rate_out,sign_rate_in,sign_rate_out,layer
0,611,95,607,98,0.925659,0.246094,0.501642,0.492228,dense1
1,596,110,608,80,0.926514,0.257812,0.495017,0.578947,dense1
2,597,93,537,98,0.930786,0.253906,0.526455,0.486911,dense1
3,578,98,585,92,0.929016,0.257812,0.496991,0.515789,dense1
4,590,105,603,99,0.927185,0.203125,0.494552,0.514706,dense1


In [24]:
sign_distr_random.tail()  

,pos_in,pos_out,neg_in,neg_out,prune_rate_in,prune_rate_out,sign_rate_in,sign_rate_out,layer
251,90,6,96,4,0.273438,0.0,0.483871,0.600000,dense2
252,98,4,99,6,0.230469,0.0,0.497462,0.400000,dense2
253,89,6,94,3,0.285156,0.1,0.486339,0.666667,dense2
254,100,6,83,2,0.285156,0.2,0.546448,0.750000,dense2
255,104,4,88,4,0.250000,0.2,0.541667,0.500000,dense2
